# Preprocessing the data

The MERFISH mouse ileum dataset contains two important large files, molecules.csv and poly\_per\_z.json, which we first want to load into the SQL container with a spatial data type column. Unsurprisingly, the data comes essentially in a text format, and needs to be preprocessed. 

## Molecules.csv

We tried multiple approaches to this, using a python script and using SQL queries. It turned out the SQL queries were simpler and easier to execute.

In [1]:
CREATE DATABASE MouseIleum;

Commands completed successfully.

Total execution time: 00:00:02.239

In [2]:
USE MouseIleum;
DROP TABLE IF EXISTS [dbo].[Molecules];
CREATE TABLE [dbo].[Molecules](
  molecule_id int,
  gene nvarchar(8),
  x_pixel smallint,
  y_pixel smallint,
  z_pixel float,
  x_um float,
  y_um float,
  z_um float,
  area tinyint,
  total_mag float,
  brightness float, 
  qc_score float
);

BULK INSERT [dbo].[Molecules] FROM '/var/data/raw_data/molecules.csv'
WITH ( 
    FIRSTROW = 2, -- skip the column headers
    ROWS_PER_BATCH = 819665, -- however many total rows the data has
    FIELDTERMINATOR = ',', 
    ROWTERMINATOR = '0x0a',
    KEEPNULLS
);

(819665 rows affected)

Total execution time: 00:00:42.516

The above code is also duplicated in importMolecules.sql.  
From here, run the processMoleculesPoints.sql script shown below.

In [3]:
USE MouseIleum;

DROP TABLE IF EXISTS MoleculesWithPoints;
SELECT molecule_id, gene, x_pixel, y_pixel, z_pixel INTO MoleculesWithPoints FROM Molecules;

ALTER TABLE MoleculesWithPoints
    ADD z_layer tinyint,
        point geometry;

UPDATE MoleculesWithPoints
    SET z_layer = CAST(ROUND(z_pixel /13.76819064, 0) AS int) + 1;

UPDATE MoleculesWithPoints
    SET point = geometry::STGeomFromText('POINT(' + CONVERT(VARCHAR(5), x_pixel) + ' ' + CONVERT(VARCHAR(5), y_pixel) + ')', 0)

(819665 rows affected)

(819665 rows affected)

(819665 rows affected)

Total execution time: 00:02:07.620

We now have a useful intermediate table MoleculesWithPoints.

# Preprocessing the polygon data

Just run the python script, get\_baysor\_polygons.py. 

Notes:

- May require some modification of the script for the file paths.
- Make sure pandas, tqdm are installed
- After running the script, move the generated csv to the processed\_data directory

In [5]:
USE MouseIleum;

DROP TABLE IF EXISTS CellPolygons;
CREATE TABLE CellPolygons (
    id int, 
    z tinyint,
    cell smallint,
    polygon_string NVARCHAR(MAX),
);

BULK INSERT CellPolygons FROM '/var/data/baysor_SQL_polygons.csv' WITH ( FIRSTROW = 2, FIELDTERMINATOR = ',');

-- SELECT * FROM CellPolygons

(33861 rows affected)

Total execution time: 00:00:17.649

In [6]:
USE MouseIleum;
UPDATE [dbo].[CellPolygons]
    SET polygon_string = REPLACE(polygon_string, '"', '')

(33861 rows affected)

Total execution time: 00:00:22.993

The above commands are duplicated in importCellPolygons.sql.

Then, run processCellPolygons.sql (below) to get the polygons represented as geometries, not strings.

In [7]:
USE MouseIleum;

ALTER TABLE CellPolygons
ADD polygon geometry;

GO
UPDATE CellPolygons
    SET polygon = geometry::STGeomFromText(polygon_string, 0);

ALTER TABLE CellPolygons
    DROP COLUMN polygon_string;

Commands completed successfully.

(33861 rows affected)

Total execution time: 00:00:37.952

Now the data is in SQL, which is great.  

# Performing spatial queries

Before we can actually do spatial stuff effectively, we should add spatial indices. 

## Adding spatial indices

Before we can add spatial indices we also have to add primary keys.

In [8]:
USE MouseIleum;
ALTER TABLE MoleculesWithPoints
ADD id int identity(1,1) not null; -- instead of adding a new column, selecting into a new table would be better (on principle)

GO
ALTER TABLE MoleculesWithPoints
ADD CONSTRAINT molecules_id_primary_key PRIMARY KEY(id);

GO
CREATE SPATIAL INDEX PointInd ON
   [MouseIleum].[dbo].[MoleculesWithPoints](point)
   WITH (GRIDS = (HIGH, HIGH, HIGH, HIGH), 
        BOUNDING_BOX = (XMIN = 112,YMIN = 0,XMAX = 5722, YMAX = 9393)); -- the actual min/max values in the data

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:01:06.893

In [9]:
USE MouseIleum
ALTER TABLE CellPolygons
ALTER COLUMN 
    id int NOT NULL;

GO
ALTER TABLE CellPolygons
ADD CONSTRAINT polygons_id_primary_key PRIMARY KEY(id);

GO
CREATE SPATIAL INDEX PolygonInd ON
   [MouseIleum].[dbo].[CellPolygons](polygon)
   WITH (GRIDS = (HIGH, HIGH, HIGH, HIGH), 
        BOUNDING_BOX = (XMIN = 112,YMIN = 0,XMAX = 5722, YMAX = 9393)); -- the actual min/max values in the data

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:43.806

## Actually running the queries

In [10]:
USE MouseIleum;
UPDATE CellPolygons
    SET polygon = polygon.MakeValid();

(33861 rows affected)

Total execution time: 00:00:23.220

In [55]:
DROP TABLE querytime;
CREATE TABLE querytime (time DATETIME);

Commands completed successfully.

Total execution time: 00:00:00.031

In [57]:
-- Set variables to track the time 
DECLARE @startTime DATETIME
declare @endTime DATETIME
declare @diff DATETIME


Set @starttime = getdate()

-- Find molecules in each cell for one layer
USE MouseIleum;
DROP TABLE IF EXISTS MoleculeCountsLayer6;
SELECT mol.gene, COUNT(mol.molecule_id) as molecule_count, poly.cell, 6 as z_layer INTO MoleculeCountsLayer6 FROM (
    SELECT * FROM [MouseIleum].[dbo].[MoleculesWithPoints] 
        WHERE z_layer=6 ) as mol
    INNER JOIN (   
        SELECT * FROM [MouseIleum].[dbo].[CellPolygons] 
        WHERE z=6 ) as poly
    ON poly.polygon.STIntersects(mol.point) = 1 -- =1 is needed bc output is 0 or 1 instead of T/F
    GROUP BY mol.gene , poly.cell;

-- Set the ending time
Set @endTime = GETDATE()
Set @diff = @endTime - @startTime
-- SELECT @EndTime - @StartTime
-- SELECT *
-- INTO #timing
-- FROM @querytime
INSERT INTO querytime VALUES (@diff)
GO 100

SELECT *
FROM querytime


Beginning execution loop

(35620 rows affected)

(1 row affected)

(35620 rows affected)

(1 row affected)

...

(1 row affected)

Batch execution completed 100 times...

(200 rows affected)

Total execution time: 00:20:35.876

time
1900-01-01 00:00:12.537
1900-01-01 00:00:13.470
1900-01-01 00:00:12.257
1900-01-01 00:00:12.413
1900-01-01 00:00:12.957
1900-01-01 00:00:13.350
1900-01-01 00:00:13.807
1900-01-01 00:00:13.500
1900-01-01 00:00:13.043
1900-01-01 00:00:13.423


In [28]:
DECLARE @startTime DATETIME
declare @endTime DATETIME
Set @starttime = getdate()

-- Find molecules in each cell for 1 z layer
-- TODO: rename this table to make more sense
USE MouseIleum;
DROP TABLE IF EXISTS MoleculeCountsAllLayers;
SELECT mol.molecule_id, mol.gene, mol.point, poly.cell as cell_id, poly.polygon as cell_polygon, 1 as z_layer INTO MoleculeCountsAllLayers FROM (
    SELECT * FROM [MouseIleum].[dbo].[MoleculesWithPoints] 
        WHERE z_layer=1 ) as mol
    INNER JOIN (   
        SELECT * FROM [MouseIleum].[dbo].[CellPolygons] 
        WHERE z=1 ) as poly
    ON poly.polygon.STIntersects(mol.point) = 1 -- =1 is needed bc output is 0 or 1 instead of T/F

Set @endTime = GETDATE()
SELECT @EndTime - @StartTime;

GO 1

(93131 rows affected)

(1 row affected)

Total execution time: 00:02:38.236

(No column name)
1900-01-01 00:02:34.030


# Segmentation & Convex Hulls

1\. Import the segmentation data using importSegmentation.sql

2\. Process into geometry and columns we care about with processSegmentationPoints.sql

Then we messed around a bit trying to figure out how this data is related to the data we have....

In [6]:
USE MouseIleum;

-- Select rows from a Table or View '[SegmentationWithPoints]' in schema '[dbo]'
SELECT TOP(10) count(mol_id) as total_molecules, cell_id FROM [dbo].[SegmentationWithPoints]
GROUP BY cell_id
ORDER BY cell_id ASC
GO

-- Select rows from a Table or View '[MoleculeCountsAllLayers]' in schema '[dbo]'
SELECT TOP(10) count(molecule_id) as total_molecules, cell_id FROM [dbo].[MoleculeCountsAllLayers]
GROUP BY cell_id
ORDER BY cell_id ASC
GO

(10 rows affected)

(10 rows affected)

Total execution time: 00:00:00.256

total_molecules,cell_id
39,1
165,2
139,3
80,4
75,5
167,6
193,7
24,8
97,9
20,10


total_molecules,cell_id
9,1
28,2
14,4
14,5
5,6
24,7
43,8
12,9
30,10
22,11


We can ignore cell 0 (those are artifacts). The rest of them don't seem to correspond.

Let's clean up the SegmentationWithPoints table by dropping any columns that we don't need, and removing molecules where is\_noise = true.

In [7]:
DELETE FROM SegmentationWithPoints WHERE is_noise='true';

ALTER TABLE SegmentationWithPoints
    DROP COLUMN gene, x_pixel, y_pixel, id, assignment_confidence, is_noise;

: Msg 207, Level 16, State 1, Line 1
Invalid column name 'is_noise'.

Total execution time: 00:00:00.003

It's time to compute some convex hulls.

In [36]:
-- Creating a table of just the first cell in z layer 1
DROP TABLE IF EXISTS #SegmentationZ1C1
SELECT xy_point 
    INTO #SegmentationZ1C1
    FROM SegmentationWithPoints
    WHERE z_layer=1 AND cell_id=1;
-- Computing the convex hull around the molecules within that cell
SELECT geometry::ConvexHullAggregate(xy_point) FROM #SegmentationZ1C1;

(8 rows affected)

(1 row affected)

Total execution time: 00:00:00.201

(No column name)
0x00000000010407000000000000000070A0400000000000002A4000000000006EA0400000000000003040000000000038A04000000000000035400000000000CC9F4000000000000039400000000000F89F40000000000000224000000000003AA040000000000000F03F000000000070A0400000000000002A4001000000020000000001000000FFFFFFFF0000000003


As far as we can tell at the moment, we have to put things into a table of some kind, since ConvexHullAggregate only works on a geometry column in an established table (that is, we cannot simply put the select statement as the argument to ConvexHullAggregate). 

Because of the necessity of creating temporary tables like this, it isn't very generalizable this way. There must be some better way to do this that we aren't seeing at the moment.

Let's try this again using a loop instead

In [18]:
DROP TABLE IF EXISTS ConvexHullsZ1;

Commands completed successfully.

Total execution time: 00:00:00.005

In [19]:
CREATE TABLE ConvexHullsZ1(
    cell_id int,
    hull geometry
    );

--DROP TABLE IF EXISTS ConvexHullsZ1;

Commands completed successfully.

Total execution time: 00:00:00.010

In [ ]:
DROP TABLE querytime;
CREATE TABLE querytime (time DATETIME);


In [22]:
Use MouseIleum
DECLARE @maxId INTEGER;
SET @maxId = 5800;
-- Set variables to track the time 
DECLARE @startTime DATETIME
declare @endTime DATETIME
declare @diff DATETIME


Set @starttime = getdate()

--SELECT MAX(cell_id) AS @maxId
--    FROM SegmentationWithPoints; 

SET NOCOUNT ON;
DECLARE @i INTEGER;
SET @i = 1;
-- WHILE @i < @maxId
WHILE @i <= @maxId
    BEGIN
        -- Creating a table of just the first cell in z layer 1
        DROP TABLE IF EXISTS #SegmentationZ3Cell;
        SELECT xy_point 
            INTO #SegmentationZ3Cell
            FROM SegmentationWithPoints
            WHERE z_layer=3 AND cell_id= @i;
        -- Computing the convex hull around the molecules within that cell
        INSERT INTO ConvexHullsZ3
            SELECT @i AS cell_id, geometry::ConvexHullAggregate(xy_point) AS hull FROM #SegmentationZ3Cell;
        SET @i = @i + 1;
    END;
SET NOCOUNT OFF;

-- Set the ending time
Set @endTime = GETDATE()
Set @diff = @endTime - @startTime

INSERT INTO querytime VALUES (@diff)

GO 10


Commands completed successfully.

Total execution time: 00:15:50.713

Now that we've created some convex hulls, let's check a few of them out:

In [5]:
USE MouseIleum
SELECT cell_id, hull.STAsText() as hull_string FROM ConvexHullsZ1;

(11600 rows affected)

Displaying Top 5000 rows.

Total execution time: 00:00:00.446

cell_id,hull_string
1,"POLYGON ((2104 13, 2103 16, 2076 21, 2035 25, 2046 9, 2077 1, 2104 13))"
2,"POLYGON ((2193 56, 2170 102, 2130 98, 2101 43, 2100 23, 2108 1, 2165 1, 2186 3, 2191 30, 2193 56))"
3,"POLYGON ((2101 88, 2087 106, 2078 114, 2059 126, 2055 68, 2079 53, 2098 67, 2101 88))"
4,"POLYGON ((2111 142, 2109 178, 2102 190, 2092 192, 2081 187, 2069 170, 2066 161, 2056 127, 2099 130, 2111 142))"
5,"POLYGON ((2275 10, 2263 24, 2237 37, 2197 1, 2220 0, 2237 2, 2275 10))"
6,"POLYGON ((2269 78, 2263 99, 2227 122, 2206 117, 2203 88, 2205 62, 2219 49, 2264 55, 2269 78))"
7,"POLYGON ((2241 180, 2234 197, 2194 215, 2183 219, 2160 201, 2144 135, 2153 114, 2190 116, 2225 126, 2231 131, 2241 180))"
8,"LINESTRING (2070 232, 2065 233)"
9,"POLYGON ((2351 3, 2341 61, 2332 63, 2298 41, 2292 28, 2298 1, 2314 0, 2351 3))"
10,NULL
